In [ ]:
import random
import string
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
from shapely.geometry import Point

In [ ]:
synthetic_data_folder = 'synthetic_data'

In [ ]:
# Generate table of towers
np.random.seed(100)
num_towers = 100

shapefile = gpd.read_file(synthetic_data_folder + '/regions.geojson')
shapefile = shapefile.dissolve(by='OBJECTID')
polygon = shapefile['geometry'][0]

minx, miny, maxx, maxy = polygon.bounds
lats, lons = [], []
while len(lats) < num_towers:
    point = Point(np.random.uniform(minx, maxx), np.random.uniform(miny, maxy))
    if polygon.contains(point):
        lats.append(point.y)
        lons.append(point.x)
points = pd.DataFrame([lats, lons]).T
points.columns = ['latitude', 'longitude']
points['tower_id'] = range(len(points))
points['tower_id'] = points['tower_id'].apply(lambda x: 't' + str(x))
points.to_csv(synthetic_data_folder + '/towers.csv', index=False)

In [ ]:
# Generate table of antennas
np.random.seed(200)
num_extra_antennas = 10

antennas = []
towers = pd.read_csv(synthetic_data_folder + '/towers.csv')
towerids = towers['tower_id'].values
for towerid in towerids:
    num_antennas = np.random.randint(1, 6)
    for i in range(num_antennas):
        antennas.append(towerid)
antennas = pd.DataFrame(antennas)
antennas.columns = ['tower_id']
antennas['antenna_id'] = range(len(antennas))
antennas['antenna_id'] = antennas['antenna_id'].apply(lambda x: 'a' + str(x))
antennas = antennas[['antenna_id', 'tower_id']]
max_antenna = int(antennas['antenna_id'].values[-1][1:])
antennas_notower = ['a' + str(i) for i in range(max_antenna+1, max_antenna+1+num_extra_antennas)]
antennas_notower = pd.DataFrame([antennas_notower, [np.nan for _ in antennas_notower]]).T
antennas_notower.columns = ['antenna_id', 'tower_id']
antennas = pd.concat([antennas, antennas_notower])
antennas.to_csv(synthetic_data_folder + '/antennas.csv', index=False)

In [ ]:
# Generate table of subscribers
num_subscribers = 1000
strlen = 10
random.seed(300)

subs = []
for i in range(num_subscribers):
    subs.append(''.join(random.choice(string.ascii_lowercase + string.ascii_uppercase) for _ in range(strlen)))
subs = pd.DataFrame(subs)
subs.columns = ['subscriber_id']
subs.to_csv(synthetic_data_folder + '/subscribers.csv', index=False)

In [ ]:
# Generate call/SMS table
np.random.seed(400)
num_txns = 100000

subs = pd.read_csv(synthetic_data_folder + '/subscribers.csv')['subscriber_id'].values.flatten()
antenna_dict = pd.read_csv(synthetic_data_folder + '/antennas.csv')
antennas = antenna_dict['antenna_id'].values.flatten()
towers = pd.read_csv(synthetic_data_folder + '/towers.csv')

start = pd.to_datetime('2020-01-01 00:00:00')
end = pd.to_datetime('2020-03-01 00:00:00')
seconds = ((end - start).days)*24*60*60

txns = []
while len(txns) < num_txns:
    caller = np.random.choice(subs)
    recipient = np.random.choice(subs)
    caller_antenna = np.random.choice(antennas)
    recipient_antenna = np.random.choice(antennas)
    international = np.random.choice(['domestic', 'international', 'other'], p=[.98, .01, .01])
    if np.random.randint(0, 10) == 0:
        caller_antenna = np.nan
    if np.random.randint(0, 10) == 0:
        recipient_antenna = np.nan
    timedif = np.random.randint(0, seconds)
    timestamp = start + pd.Timedelta(seconds=timedif)
    if caller != recipient:
        txns.append([caller, recipient, caller_antenna, recipient_antenna, timestamp, international])

txns = pd.DataFrame(txns)
txns.columns = ['caller_id', 'recipient_id', 'caller_antenna', 'recipient_antenna', 'timestamp', 'international']
txns['interaction'] = np.random.choice(['voice', 'sms'], size=len(txns))
txns['duration'] = np.random.randint(0, 300, size=len(txns))
txns['duration'] = txns.apply(lambda row: row['duration'] if row['interaction'] == 'voice' else np.nan, axis=1)
txns = txns.sort_values('timestamp', ascending=True)
txns = txns[['interaction', 'caller_id', 'recipient_id', 'timestamp', 'duration', 'caller_antenna', 
             'recipient_antenna', 'international']]

txns.to_csv(synthetic_data_folder + '/cdr.csv', index=False)

In [ ]:
txns.head()

In [ ]:
# Generate mobile data usage table
np.random.seed(500)
num_data_txns = 10000
percent_use_data = 0.6

subs = pd.read_csv(synthetic_data_folder + '/subscribers.csv')
subs = subs.sample(frac=1, replace=False)[:int(len(subs)*percent_use_data)]
mobiledata = subs.sample(num_data_txns, replace=True)
mobiledata['volume'] = np.random.normal(100, 20, num_data_txns)

start = pd.to_datetime('2020-01-01 00:00:00')
end = pd.to_datetime('2020-03-01 00:00:00')
seconds = ((end - start).days)*24*60*60
timestamps = [start + pd.Timedelta(seconds=np.random.randint(0, seconds)) for _ in range(num_data_txns)]
mobiledata['timestamp'] = timestamps
mobiledata = mobiledata.sort_values('timestamp', ascending=True)

mobiledata.to_csv(synthetic_data_folder + '/mobiledata.csv', index=False)

In [ ]:
mobiledata.head()

In [ ]:
# Generate top-ups table
np.random.seed(600)
num_topup_txns = 10000
percent_use_topups = 0.8

subs = pd.read_csv(synthetic_data_folder + '/subscribers.csv')
subs = subs.sample(frac=1, replace=False)[:int(len(subs)*percent_use_topups)]
topups = subs.sample(num_topup_txns, replace=True)
topups['amount'] = np.random.randint(0, 100, num_topup_txns)

start = pd.to_datetime('2020-01-01 00:00:00')
end = pd.to_datetime('2020-03-01 00:00:00')
seconds = ((end - start).days)*24*60*60
timestamps = [start + pd.Timedelta(seconds=np.random.randint(0, seconds)) for _ in range(num_data_txns)]
topups['timestamp'] = timestamps
topups = topups.sort_values('timestamp', ascending=True)

topups.to_csv(synthetic_data_folder + '/topups.csv', index=False)

In [ ]:
topups.head()

In [ ]:
# Generate mobile money table
np.random.seed(700)
num_mm_txns = 10000
percent_use_mm = 0.7
txn_types = ['cashin', 'cashout', 'p2p', 'billpay', 'other']
txn_probs = [.2, .2, .4, .1, .1]

subs = pd.read_csv(synthetic_data_folder + '/subscribers.csv')
subs = subs.sample(frac=1, replace=False)[:int(len(subs)*percent_use_mm)]
subs = subs['subscriber_id'].values.flatten()

start = pd.to_datetime('2020-01-01 00:00:00')
end = pd.to_datetime('2020-03-01 00:00:00')
seconds = ((end - start).days)*24*60*60

mm_txns = []
while len(mm_txns) < num_mm_txns:
    txn_type = np.random.choice(txn_types, p=txn_probs)
    caller = np.random.choice(subs)
    if txn_type in ['p2p', 'billpay', 'other']:
        recipient = np.random.choice(subs)
    else:
        recipient = np.nan
    timedif = np.random.randint(0, seconds)
    timestamp = start + pd.Timedelta(seconds=timedif)
    if caller != recipient:
        mm_txns.append([txn_type, caller, recipient, timestamp])

mm_txns = pd.DataFrame(mm_txns, columns = ['txn_type', 'sender', 'recipient', 'timestamp'])
mm_txns['amount'] = np.random.normal(50, 10, num_mm_txns)
#mm_txns['amount'] = mm_txns\
#    .apply(lambda row: -row['amount'] if row['txn_type'] == 'cashout' 
#           else row['amount'], axis=1)
mm_txns['sender_balance_before'] = np.random.normal(200, 40, num_mm_txns)
mm_txns['sender_balance_after'] = mm_txns\
    .apply(lambda row: row['sender_balance_before'] + row['amount'] if row['txn_type'] == 'cashin'
          else row['sender_balance_before'] - row['amount'], axis=1)
mm_txns['recipient_balance_before'] = np.random.normal(200, 40, num_mm_txns)
mm_txns['recipient_balance_before'] = mm_txns\
    .apply(lambda row: np.nan if row['txn_type'] in ['cashin', 'cashout'] else row['recipient_balance_before'],
          axis=1)
mm_txns['recipient_balance_after'] = mm_txns['recipient_balance_before'] + mm_txns['amount']
mm_txns = mm_txns.sort_values('timestamp', ascending=True)

mm_txns.to_csv(synthetic_data_folder + '/mobilemoney.csv', index=False)

In [ ]:
np.random.seed(1)
feats = pd.read_csv('../synthetic_outputs/datasets/features.csv')
feats['label'] = feats['cdr_reporting__number_of_records']**2 + \
    feats['cdr_active_days__weekend__day__callandtext']**3 + \
    feats['location_count(regions)']**5
feats['label'] = feats['label'] + np.random.randint(0, 50000, len(feats))
feats = feats[['name', 'label']].sample(50, replace=True, random_state=2)
feats['weight'] = (np.random.rand(1, len(feats))*100).flatten()
feats.to_csv('../synthetic_data/labels.csv', index=False)


In [ ]:
subscribers = pd.read_csv('../synthetic_data/subscribers.csv')
antennas = pd.read_csv('../synthetic_data/antennas.csv')
regions = gpd.read_file('../synthetic_data/regions.geojson')
prefectures = gpd.read_file('../synthetic_data/prefectures.geojson')
subscribers['antenna_id'] = antennas['antenna_id'].drop_duplicates()\
    .sample(len(subscribers), replace=True).values
subscribers['tower_id'] = antennas['tower_id'].drop_duplicates()\
    .sample(len(subscribers), replace=True).values
subscribers['regions'] = regions['region'].drop_duplicates()\
    .sample(len(subscribers), replace=True).values
subscribers['prefectures'] = prefectures['region'].drop_duplicates()\
    .sample(len(subscribers), replace=True).values
subscribers.to_csv('../synthetic_data/home_locations.csv', index=False)

### Survey data

In [ ]:
np.random.seed(100)
obs = 1000

# Unique IDs
uids = np.arange(obs)

# Phone numbers
strlen = 10
subs = []
for i in range(obs):
    subs.append(''.join(random.choice(string.ascii_lowercase + string.ascii_uppercase) for _ in range(strlen)))
    
# Consumption
consumption = np.random.rand(obs)*50

# Binary questions
binary_questions = 10
binary_data = pd.DataFrame([np.round(np.random.rand(obs)) for i in range(1, 1+binary_questions)]).T
binary_columns = ['bin' + str(i) for i in range(binary_questions)]
binary_data.columns = binary_columns

# Continuous questions
continuous_questions = 10
continuous_data = pd.DataFrame([np.random.rand(obs)*i for i in range(1, 1+continuous_questions)]).T
continuous_columns = ['con' + str(i) for i in range(continuous_questions)]
continuous_data.columns = continuous_columns

# Categorical questions
categorical_questions = 10
categorical_data = pd.DataFrame([np.random.randint(low=0, high=10, size=obs) for i in \
                                 range(1, 1+categorical_questions)]).T
categorical_columns = ['cat' + str(i) for i in range(categorical_questions)]
categorical_data.columns = categorical_columns

# Combine data
data = pd.concat([binary_data, continuous_data, categorical_data], axis=1)
data['unique_id'] = uids
data['consumption'] = consumption
data['weight'] = np.random.rand(obs)*10
data['phone_number'] = subs
data = data[['unique_id', 'phone_number', 'weight', 'consumption'] + binary_columns + continuous_columns + \
            categorical_columns]

# Add some missing data
for col in ['consumption'] + binary_columns + categorical_columns + continuous_columns:
    data[col] = data[col].apply(lambda x: np.nan if np.random.rand() < .01 else x)

# Save to file
data.to_csv('survey.csv', index=False)

In [ ]:
data.head()